In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import plotnine as pn
from plotnine import ggplot, geom_point, aes, geom_col, geom_line, geom_boxplot, geom_bar
from scipy.stats import ttest_ind
pd.set_option('max_rows', 200)
pd.set_option('max_colwidth', 150)

In [2]:
import psycopg2
import pandas.io.sql as sqlio

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
conn = psycopg2.connect(dbname='quasar_prod_warehouse', user='kkennovin', password='eBC$O55$Lcko', host='quasar-prod.c9ajz690mens.us-east-1.rds.amazonaws.com', port='5432')

In [4]:

sql_q1 = """select 
	northstar_id
,	text 
from 
	posts 
where 
	campaign_id = '9115' and 
	action_id = 1126
    """

sql_q2 = """select 
	northstar_id
,	text 
from 
	posts 
where 
	campaign_id = '9115' and 
	action_id = 1127
    """

sql_q3 = """select 
	northstar_id
,	text 
from 
	posts 
where 
	campaign_id = '9115' and 
	action_id = 1128
    """

In [5]:
data_q1 = sqlio.read_sql_query(sql_q1, conn)
data_q2 = sqlio.read_sql_query(sql_q2, conn)
data_q3 = sqlio.read_sql_query(sql_q3, conn)

In [6]:
conn = None

In [8]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove punctuation'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# Let's take a look at the updated text
data_clean_q1 = pd.DataFrame(data_q1.text.apply(round1))
#data_clean_q1.sample(50)
data_clean_q2 = pd.DataFrame(data_q2.text.apply(round1))
#data_clean_q2.sample(50)
data_clean_q3 = pd.DataFrame(data_q3.text.apply(round1))
#data_clean_q3.sample(50)

# looks good! I don't see any other obvious cleaning needs

In [22]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words for each question
from sklearn.feature_extraction.text import CountVectorizer

cv_q1 = CountVectorizer(stop_words='english')
data_cv_q1 = cv_q1.fit_transform(data_clean_q1.text)
dtm_q1 = pd.DataFrame(data_cv_q1.toarray(), columns=cv_q1.get_feature_names())
dtm_q1.index = data_clean_q1.index

cv_q2 = CountVectorizer(stop_words='english')
data_cv_q2 = cv_q2.fit_transform(data_clean_q2.text)
dtm_q2 = pd.DataFrame(data_cv_q2.toarray(), columns=cv_q2.get_feature_names())
dtm_q2.index = data_clean_q2.index

cv_q3 = CountVectorizer(stop_words='english')
data_cv_q3 = cv_q3.fit_transform(data_clean_q3.text)
dtm_q3 = pd.DataFrame(data_cv_q3.toarray(), columns=cv_q3.get_feature_names())
dtm_q3.index = data_clean_q3.index
